# edgedb

testing the edgedb database from rust

In [9]:
// import package dependencies
:dep color-eyre = "0.6.2"
:dep uuid = { version = "1.3.0", features = ["v4", "macro-diagnostics", "fast-rng", "serde"] }
:dep serde = "1.0.189"
:dep serde_json = "1.0.107"
:dep tokio = { version = "1.33.0", features = ["full"] }
:dep edgedb-tokio = "0.5.0"
:dep edgedb-protocol = "0.6.0"
:dep edgedb-derive = "0.5.1"

## edgedb-cli

#### setup

create a new project in the root directory of the code repository by running the following commands. Note that edgedb shoul already be running in a container using docker, kubernetes or any other containerised tool.

```sh
# list all edgedbb instances
❯ edgedb instance list
┌────────┬───────────┬─────────────────┬──────────────┬──────────┐
│ Kind   │ Name      │ Location        │ Version      │ Status   │
├────────┼───────────┼─────────────────┼──────────────┼──────────┤
│ local  │ labs      │ localhost:10705 │ 2.12+f3129ef │ inactive │
│ local  │ tenchi    │ localhost:10706 │ 2.12+f3129ef │ inactive │
│ remote │ localhost │ localhost:5656  │ 3.4+7598b99  │ up       │
└────────┴───────────┴─────────────────┴──────────────┴──────────┘
```

instantiate a new project (choose remote instance name e.g. localhost)
```sh
# instantiate a new project
❯ edgedb project init
```

check that the project is connected to the instance

```sh
❯ edgedb project info
┌───────────────┬────────────────────────────────────────────┐
│ Instance name │ localhost                                  │
│ Project root  │ /Users/goose/Spaces/pepeye/github/ai/mlops │
└───────────────┴────────────────────────────────────────────┘
```

#### migrations

```sh
# restore database
❯ edgedb restore -I localhost path/to/dump/file/edgedb.dump
```

## dependencies

import package dependencies

In [10]:
// import dependencies
use color_eyre::Result;
use edgedb_tokio::create_client;
use edgedb_tokio::Client;
use edgedb_derive::Queryable;
use serde::{Serialize, Deserialize};
use serde_json::json;
use uuid::Uuid;

## connection

In [11]:
// connect to database
let client = create_client()
    .await
    .expect("Client should have initiated");

// query
let val: i64 = client
    .query_required_single("select 7*8", &())
    .await
    .expect("Query should have worked");

// print resulting value
println!("[out]: 7*8 is: {val}");

// Without arguments: just add &() after the query
let query_a: String =
    client.query_required_single("select 'Just a string'", &()).await?;

// With arguments, same output as the previous example
let a = " a ";
let b = "string";
let query_b: String = client
    .query_required_single("select 'Just' ++ <str>$0 ++ <str>$1", &(a, b))
    .await?;

// print resulting value
println!("[out]: {}", query_a);
println!("[out]: {}", query_b);

[out]: 7*8 is: 56
[out]: Just a string
[out]: Just a string


## queries

edgedb query to insert a new person into the database. The data payload below is used as the variable `$0`.
```
with data := <json>$0
insert Person {
  firstname := <str>data['firstname'],
  lastname :=  <str>data['lastname'],
  nickname := <str>data['nickname'],
  status :=  <Status>data['status'],
  identity := (insert Identity {
    email := <str>data['email'],
    login := <str>data['login'],
    password := <str>data['password'],
    preferred := <bool>data['preferred'],
    provider := (select (
      insert Provider { name := <optional str>data['provider'] }
      unless conflict on .name 
      else 
        (select Provider filter .name = <optional str>data['provider'] Limit 1)
    ))
  })
}
```

### request payload

The following data payload will be used to create a record in edgedb and used for retrieval purposes
```json
{
  "firstname": "tony",
  "lastname": "stark",
  "nickname": "ironman",
  "status": "PENDING",
  "email": "ironman@avengers.com",
  "login": "@ironman",
  "password": "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He",
  "preferred": true,
  "provider": "ardency"
}
```

## models

In [12]:
#[derive(Debug, Serialize, Deserialize, Queryable)]
pub struct Person {
    id: Uuid,
    firstname: String,
    lastname: String,
    fullname: String,
    nickname: String,
    status: String,
    identity: Vec<Identity>,
    updated: String,
    created: String,
}

#[derive(Debug, Serialize, Deserialize, Queryable)]
pub struct Identity {
    id: String,
    email: String,
    login: String,
    password: String,
    preferred: bool,
    provider: Provider,
}

#[derive(Debug, Serialize, Deserialize, Queryable)]
pub struct Provider {
    id: String,
    name: String,
}
#[derive(Debug, Serialize, Deserialize, Queryable)]
struct NewPerson {
  firstname: String,
  lastname: String,
  nickname: String,
  status: String,
  email: String,
  login: String,
  password: String,
  preferred: bool,
  provider: String,
}

#[derive(Debug, Queryable, Deserialize)]
pub struct QueryResult {
    pub id: Uuid,
}

In [13]:
// create person payload
let person = NewPerson {
  firstname: "tony".to_owned(),
  lastname: "stark".to_owned(),
  nickname: "ironman".to_owned(),
  status: "PENDING".to_owned(),
  email: "ironman@avengers.com".to_owned(),
  login: "@ironman".to_owned(),
  password: "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He".to_owned(),
  preferred: true,
  provider: "ardency".to_owned(),
};

// create database client
let client = create_client()
    .await
    .expect("Client should have initiated");

async fn register(client: Client, person: NewPerson) -> Result<Vec<QueryResult>> {
    let q = r#"
      with data := <json>$0
      insert Person {
        firstname := <str>data['firstname'],
        lastname :=  <str>data['lastname'],
        nickname := <str>data['nickname'],
        status :=  <Status>data['status'],
        identity := (insert Identity {
          email := <str>data['email'],
          login := <str>data['login'],
          password := <str>data['password'],
          preferred := <bool>data['preferred'],
          provider := (select (
            insert Provider { name := <optional str>data['provider'] }
            unless conflict on .name 
            else 
              (select Provider filter .name = <optional str>data['provider'] Limit 1)
          ))
        })
      }
    "#;
    
    let data = serde_json::to_string(&person)?;
    let id = client.query(q, &(data,)).await?;
    Ok(id)
}

println!("[INFO]: {:#?}",person);
let id = register(client, person).await?;
id

[INFO]: NewPerson {
    firstname: "tony",
    lastname: "stark",
    nickname: "ironman",
    status: "PENDING",
    email: "ironman@avengers.com",
    login: "@ironman",
    password: "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He",
    preferred: true,
    provider: "ardency",
}


DescriptorMismatch: 
EdgeDB returned unexpected type BaseScalar(BaseScalarTypeDescriptor { id: BaseScalar(std::json) })
Client expected std::str


### select

query and retrieve data from the database

In [14]:
// make pool connection to database
let client = create_client().await?;

// define a query
let query = r#"
    select Person {
        id,
        firstname,
        lastname,
        fullname,
        nickname,
        status,
        identity: {
            id,
            email,
            login,
            password,
            preferred,
            provider: {
                id,
                name,
            },
        },
        updated,
        created,
    }
"#;

// // fetch data
let r: Vec<Person> = client.query(query, &()).await?;
r

DescriptorMismatch
